### OBTENCION DE DATOS API MERCADOLIBRE

#### Importacion de librerias necesarias

In [1]:
import json
import requests
from urllib.parse import quote
import pandas as pd

#### Obtencion de categorias

El objetivo de esta seccion es obtener un listado de las categorias y sub-categorias disponibles actualmente en la API de **MercadoLibre (https://developers.mercadolibre.com.ar/)** utilizando la libreria **requests** de python.<br>Como primer paso se hace una request a la API para obtener el listado de las categorias principales.

In [2]:
#Request para obtener las categorias principales
url = "https://api.mercadolibre.com/sites/MLA/categories"
response = requests.get(url)

De la respuesta obtenida se extraen los los ID de las categorias principales. El objetivo de esto es hacer una request por cada ID de categoria pricipal y asi obtener los datos completos de estas, incluidas las subcategorias.

In [3]:
#Se obtienen ID de las categorias principales
id_list = []

#Ciclo for para extraer el campo "id" de cada categoria
if response.status_code == 200:
    data = response.json()
    for item in data:
        id_list.append(item['id'])    
else:
    print(response.status_code)

print(id_list)

['MLA5725', 'MLA1512', 'MLA1403', 'MLA1071', 'MLA1367', 'MLA1368', 'MLA1743', 'MLA1384', 'MLA1246', 'MLA1039', 'MLA1051', 'MLA1648', 'MLA1144', 'MLA1500', 'MLA1276', 'MLA5726', 'MLA1000', 'MLA2547', 'MLA407134', 'MLA1574', 'MLA1499', 'MLA1459', 'MLA1182', 'MLA3937', 'MLA1132', 'MLA3025', 'MLA1168', 'MLA1430', 'MLA409431', 'MLA1540', 'MLA9304', 'MLA1953']


Como se menciono anteriormente se generea una request para cada "id" de la lista **id_list** y se alojan los resultados obtenidos en la lista **categories**. El resultado obtenido es una lista de diccionarios.

In [4]:
#Request para obtener informacion y subcategorias de los id que se obtuvieron
categories = []

for id in id_list:
    url = f"https://api.mercadolibre.com/categories/{id}"
    response = requests.get(url)
    if response.status_code == 200:
        categories.append(response.json())
    else:
        print(f"error:{response.status_code} en el ID:{id}")

Se crea la clase **Category()** cuyos atributos son los datos que interesan obtener de la request anterior. Tambien se crea la clase **CategoryBuilder()** que se encarga de extraer los datos de la request y devolver una instancia de la clase **Category()** con sus respectivos atributos seteados.

In [8]:
class Category():
    def __init__(self, id, name, is_subsctegory, father_cat):
        self.id = id
        self.name = name
        self.is_subcategory = is_subsctegory
        self.father_cat = father_cat

def get_categories(data:list) -> list:
    categories=[]
    for cat in data:
        id = cat.get('id', None)
        name = cat.get('name', None)
        if 'children_categories' in cat:
            is_subcategoy = False
            father_cat = None
            category = Category(id, name, is_subcategoy, father_cat)
            categories.append(vars(category))

            for sub_cat in cat['children_categories']:
                id = sub_cat.get('id', None)
                name = sub_cat.get('name', None)
                is_subcategoy = True
                father_cat = cat.get('id', None)
                category = Category(id, name, is_subcategoy, father_cat)
                categories.append(vars(category))
        else:
            is_subcategoy = False
            father_cat = None
            category = Category(id, name, is_subcategoy, father_cat)
            categories.append(vars(category))
    return categories      

Ahora se hace uso de las clases creadas. Primero se instancia una clase **CategoryBuilder()** la cual recibe en su parametro la lista de diccionarios obtenida anteriormente(**categories**). Acto seguido se asigna un nuevo valor a **categories** utilizando el metodo **Build** a fines de obtener una lista de diccionarios con los campos de interes.

In [9]:
#Se setea la variable "categories" con una nueva lista diccionarios
categories = get_categories(categories)

Se guarda el archivo **JSON** de manera local

In [13]:
#Se guarda el archivo JSON
with open('MeliCategorias.json', 'w') as file:
    json.dump(categories, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**categories**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [14]:
df = pd.DataFrame(categories)
df.head()

,id,name,is_subcategory,father_cat
0,MLA5725,Accesorios para Vehículos,False,None
1,MLA4711,Acc. para Motos y Cuatriciclos,True,MLA5725
2,MLA417044,Accesorios Náuticos,True,MLA5725
3,MLA6520,Accesorios de Auto y Camioneta,True,MLA5725
4,MLA86360,Accesorios para Línea Pesada,True,MLA5725


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [15]:
df = pd.read_json('MeliCategorias.json')
df.head()

,id,name,is_subcategory,father_cat
0,MLA5725,Accesorios para Vehículos,False,None
1,MLA4711,Acc. para Motos y Cuatriciclos,True,MLA5725
2,MLA417044,Accesorios Náuticos,True,MLA5725
3,MLA6520,Accesorios de Auto y Camioneta,True,MLA5725
4,MLA86360,Accesorios para Línea Pesada,True,MLA5725


#### Obtencion de publicaciones

El objetivo de esta seccion es obtener un listado de publicaciones de "vehiculos" desde la API de **MercadoLibre (https://developers.mercadolibre.com.ar/)** utilizando la libreria **requests** de python.<br>En primer lugar se crea una lista(**cars**) que aloja diferentes modelos de vehiculos disponibles en el mercado automotriz, posteriormente mediante el uso de la libreria **urllib** y su metodo **quote** se codifica la lista **cars** a fin de transformar ciertos caracteres para mantener la integridad de la URL al momento de realizar la request.

In [2]:
#43 autos
cars = [    'Volkswagen gol','Volkswagen gol trend', 'Volkswagen up', 'Volkswagen bora', 'Volkswagen vento', 'Volkswagen polo', 'Volkswagen amarok',
            'Chevrolet corsa', 'Chevrolet onix', 'Chevrolet agile', 'Chevrolet prisma', 'Chevrolet cruze', 'Chevrolet aveo', 'Chevrolet tracker',
            'Renault clio', 'Renault sandero', 'Renault fluence', 'Renault logan', 'Reanault partner', 'Renault capture',
            'Peugeot 206', 'Peugeot 207', 'Peugeot 208', 'Peugeot 307','Peugeot 308', 'Peugeot 407', 'Peugeot 408',
            'Fiat cronos', 'Fiat argos', 'Fiat toro', 'Fiat palio', 'Fiat siena', 'Fiat uno',                   
            'Ford fiesta', 'Ford Ecosport', 'Ford ka', 'Ford ranger', 'Ford focus',
            'Toyota etios', 'Toyota hilux', 'Toyota corolla', 'Toyota yaris', 
            'Honda civic' ]

#Se codifican los nombres
cars_codif = []
for car in cars:
    cars_codif.append(quote(car))

Con la finalidad de realizar una request por cada modelo de automovil en la lista, se crea la funcion **get_publication()**. Esta funcion  devuelve una lista de diccionarios, cada uno de estos diccionarios representa una publicacion de **MercadoLibre**. Dentro de la logica de la funcion se establece un maximo de 250 publicaciones por vehiculo.

In [10]:
def get_publications(car : str) -> list:
    publication = []
    url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        results = data.get('paging', dict).get('total', None)  #Total de resultados existentes
        offset = 0 #Pocision de inicio de los resultados

        if 'results' in data:
            for item in data['results']:
                publication.append(item)

            while (results > offset) and (results > 50):
                offset +=50 
                if offset == 300: break 
                url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}&offset={offset}'
                response = requests.get(url)
                data = response.json()
                if 'results' in data:
                    for item in data['results']:
                        publication.append(item)  

        return publication                    
    else:
        return publication

Se crea una la lista **publications_list** la cual alojará los diccionarios obtenidos de la funcion **get_publication()**. Acto seguido ejecuta un ciclo for a fines de recorrer la lista **cars_codif**, ejecutar la funcion **get_publication()** por cada uno de los elementos en la lista, luego los resultados obtenidos son alojados en **publications_list**

In [11]:
#Lista para alojar las publicaciones
publications_list = []

for car in cars_codif:
    publications = get_publications(car)
    for publication in publications:
        publications_list.append(publication)

print(f'La cantidad de publicaciones obtenidas es: {len(publications_list)}')

La cantidad de publicaciones obtenidas es: 12196


Se guarda un archivo **JSON** con el resultado obtenido en el paso anterior

In [12]:
#GUARDADO DE JSON
with open('MeliDataPubl.json','w') as file:
    json.dump(publications_list, file, indent=4, separators=(',',':'))

#### Extraccion de los datos de interes

Con la finalidad de estructurar los datos extraidos de la API anteriormente se crea la clase **CarPublicaction()** la cual aloja en sus atributos los datos de interes de las publicaciones que se obtubieron. Tambien se crea la clase **CarPublicationBuilder()** cuya responsabilidad es extraer de un diccionario(publicacion) los datos de interes y devolver una instancia de la clase **CarPublicaction()** con sus atributos seteados

In [14]:
class CarPublication():
    def __init__(self):
        self.id = None
        self.category_id = None
        self.title = None
        self.condition = None
        self.car_year = None
        self.brand = None
        self.model = None
        self.version = None
        self.engine = None
        self.engine_power = None
        self.doors = None        
        self.km = None
        self.fuel_type = None        
        self.traction_control = None
        self.passenger_capacity = None
        self.transmission = None
        self.currency = None
        self.price = None
        self.seller_id = None
        self.seller_nickname = None
        self.is_car_shop = None
        self.seller_country = None
        self.seller_state = None
        self.seller_city = None
        self.seller_neighborhood = None


class CarPublicationBuilder():
    def __init__(self, data:dict):
        self.data = data
        self.publication = CarPublication()

    def build(self) -> CarPublication():
        self.publication.id = self._get_id()
        self.publication.category_id = self._get_category_id()
        self.publication.title = self._get_title()
        self.publication.condition = self._get_condition()
        self.publication.car_year = self._get_car_year()
        self.publication.brand = self._get_brand()
        self.publication.model = self._get_model()
        self.publication.version = self._get_version()
        self.publication.engine = self._get_engine()
        self.publication.engine_power = self._get_engine_power()
        self.publication.doors = self._get_doors()
        self.publication.km = self._get_km()
        self.publication.fuel_type = self._get_fuel_type()     
        self.publication.traction_control = self._get_traction_control()
        self.publication.passenger_capacity = self._get_passenger_capacity()
        self.publication.transmission = self._get_transmission()
        self.publication.currency = self._get_currency()
        self.publication.price = self._get_price()
        self.publication.seller_id = self._get_seller_id()
        self.publication.seller_nickname = self._get_seller_nickname()
        self.publication.is_car_shop = self._get_is_oficial_store()
        self.publication.seller_country = self._get_seller_country()
        self.publication.seller_state = self._get_seller_state()
        self.publication.seller_city = self._get_seller_city()
        self.publication.seller_neighborhood = self._get_seller_neighborhood()
        return self.publication

    def _get_id(self):
        return self.data.get('id', None)
    
    def _get_category_id(self):
        return self.data.get('category_id', None)
    
    def _get_title(self):
        return self.data.get('title', None)
    
    def _get_condition(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'ITEM_CONDITION'):
                return item['value_name']
        return None
    
    def _get_car_year(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'VEHICLE_YEAR'):
                return item['value_name']
        return None
    
    def _get_brand(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'BRAND'):
                return item['value_name']
        return None
    
    def _get_model(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'MODEL'):
                return item['value_name']
        return None
    
    def _get_version(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'TRIM'):
                return item['value_name']
        return None
    
    def _get_engine(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'ENGINE'):
                return item['value_name']
        return None
    
    def _get_engine_power(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'POWER'):
                return item['value_name']
        return None
    
    def _get_doors(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'DOORS'):
                return item['value_name']
        return None
    
    def _get_km(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'KILOMETERS'):
                return item['value_name']
        return None
    
    def _get_fuel_type(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'FUEL_TYPE'):
                return item['value_name']
        return None
    
    def _get_traction_control(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'TRACTION_CONTROL'):
                return item['value_name']
        return None
    
    def _get_passenger_capacity(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'PASSENGER_CAPACITY'):
                return item['value_name']
        return None
    
    def _get_transmission(self):
        for item in self.data['attributes']:
            if ('id' in item) and (item['id'] == 'TRANSMISSION'):
                return item['value_name']
        return None
    
    def _get_currency(self):
        return self.data.get('currency_id', None)
    
    def _get_price(self):
        return self.data.get('price', None)
    
    def _get_seller_id(self):
        return self.data.get('seller', {}).get('id', None)
    
    def _get_seller_nickname(self):
        return self.data.get('seller', {}).get('nickname', None)
    
    def _get_is_oficial_store(self):
        return self.data.get('official_store_id', None) is not None
    
    def _get_seller_country(self):
        return self.data.get('location',{}).get('country', {}).get('name', None)
    
    def _get_seller_state(self):
        return self.data.get('location',{}).get('state', {}).get('name', None)
    
    def _get_seller_city(self):
        return self.data.get('location',{}).get('city', {}).get('name', None)
    
    def _get_seller_neighborhood(self):
        return self.data.get('location',{}).get('neighborhood', {}).get('name', None)  
    

Mediante un ciclo for se recorre **publications_list** con la finalidad de ejecutar una instancia de la clase **CarPublicationBuilder()** y obtener una objeto de tipo **CarPublicaction()** por cada diccionario en **publications_list** 

In [16]:
car_publications = []

for item in publications_list:
    builder = CarPublicationBuilder(item)
    publication = builder.build()
    car_publications.append(vars(publication))

print(f'Cantidad de registros obtenidos: {len(car_publications)}')

Cantidad de registros obtenidos: 12196


Se guarda el resultado obtenido en un archivo **JSON** de manera local

In [17]:
with open('MeliCarPublicactions.json','w' ) as file:
    json.dump(car_publications, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**car_publications**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [18]:
df = pd.DataFrame(car_publications)
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1413685801,MLA1744,Volkswagen Gol 1.4 3ptas Año 2013 - Liv Motors,Usado,2013,Volkswagen,Gol,1.4 Power 83cv 3 p,1.4,83 hp,...,Manual,ARS,6000000,780050555,LIVMOTORS 2,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Munro
1,MLA1406864357,MLA1744,Volkswagen Gol 2012 1.4 Power Ps+ac 83cv,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,10500,210095605,MASSEYGROUP EXCLUSIVOS,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Olivos
2,MLA1677544030,MLA1744,Volkswagen Gol Power 3p 2012 Financiación Con Dni,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,ARS,5800000,5571520,DASAUTOS2014,False,Argentina,Bs.As. G.B.A. Sur,Berazategui,Berazategui


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [19]:
df = pd.read_json('MeliCarPublications.json')
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1413685801,MLA1744,Volkswagen Gol 1.4 3ptas Año 2013 - Liv Motors,Usado,2013,Volkswagen,Gol,1.4 Power 83cv 3 p,1.4,83 hp,...,Manual,ARS,6000000,780050555,LIVMOTORS 2,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Munro
1,MLA1406864357,MLA1744,Volkswagen Gol 2012 1.4 Power Ps+ac 83cv,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,10500,210095605,MASSEYGROUP EXCLUSIVOS,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Olivos
2,MLA1677544030,MLA1744,Volkswagen Gol Power 3p 2012 Financiación Con Dni,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,ARS,5800000,5571520,DASAUTOS2014,False,Argentina,Bs.As. G.B.A. Sur,Berazategui,Berazategui


In [20]:
print(f'Cantidad de filas y columnas: {df.shape}\n\n')

df.info()

Cantidad de filas y columnas: (12196, 25)


<class 'pandas.core.frame.DataFrame'>
Index: 12196 entries, 0 to 12195
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   12196 non-null  object 
 1   category_id          12196 non-null  object 
 2   title                12196 non-null  object 
 3   condition            12196 non-null  object 
 4   car_year             12196 non-null  int64  
 5   brand                12196 non-null  object 
 6   model                12196 non-null  object 
 7   version              12196 non-null  object 
 8   engine               11885 non-null  object 
 9   engine_power         11349 non-null  object 
 10  doors                12196 non-null  int64  
 11  km                   12196 non-null  object 
 12  fuel_type            12196 non-null  object 
 13  traction_control     11809 non-null  object 
 14  passenger_capacity   11275 non-null  float64
 1